In [3]:
import json
import pandas as pd
import io
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm_notebook, tqdm
import glob
from datetime import datetime, timedelta

In [4]:
# Define the currency
CURRENCY = "bitcoin"
CURRENCY_SYMBOL = "BTC"

CRYPTO_FOLDER = f"data/crypto/{CURRENCY_SYMBOL}"
TWITTER_FOLDER = f"data/twitter/{CURRENCY_SYMBOL}"

In [5]:
# update historical data until today
file = open("current_crypto.txt", "w") 
file.write(CURRENCY_SYMBOL)
file.close()

# Twitter Streaming

In [6]:
from twython import TwythonStreamer
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:
analyzer = SentimentIntensityAnalyzer()

In [8]:
class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        if 'text' in data:
            # x value: date
            date_str = data['created_at']
            time_struct = time.strptime(date_str, "%a %b %d %H:%M:%S +0000 %Y")
            date = datetime.fromtimestamp(time.mktime(time_struct))

            text = data['text']
            vs = analyzer.polarity_scores(text)
            
            score = vs["compound"] * (data["user"]["followers_count"]+1) * (data["favorite_count"]+1) * (data["retweet_count"]+1)
            file=open("streamdata.txt","a")
            file.write(repr(score)+",")
            print(date, repr(score))

    def on_error(self, status_code, data):
        print(status_code, data)

        # Want to stop trying to get data because of the error?
        # Uncomment the next line!
        # self.disconnect()

In [9]:
APP_KEY = 'mPQKoRwd2Pb9qpQyQmyG5s8KR'
APP_SECRET = 'HLvIhusvfzDLKaRXY8CnZGP143kp3E3f2KqQBIEMfVL5mOxZjq'
OAUTH_TOKEN = "3459248236-0XPtHldG3ou6BfpTwaKWnOL2ywFk2niQekLwE7K"
OAUTH_TOKEN_SECRET = "08Vy2wuOkp7AmuC3rbjCHFJ94MLG2sWqdvGQtoiXmkVKr"

In [10]:
stream = MyStreamer(APP_KEY, APP_SECRET,
                    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [11]:
stream.statuses.filter(track=['#bitcoin', '#BTC'], lang='en')